In [14]:
from pandas import *
# from numpy import *
import pandas as pd
# import numpy as np
from scipy.stats import *


In [15]:
# get data from https://www.football-data.co.uk/
data = pd.read_csv('https://www.football-data.co.uk/mmz4281/1516/E0.csv')

data = data[['Date','HomeTeam','AwayTeam','FTHG','FTAG']]


In [16]:
lg_games = len(data['HomeTeam'].unique())
data['HomeTeam'].unique()

array(['Bournemouth', 'Chelsea', 'Everton', 'Leicester', 'Man United',
       'Norwich', 'Arsenal', 'Newcastle', 'Stoke', 'West Brom',
       'Aston Villa', 'Southampton', 'Sunderland', 'Swansea', 'Tottenham',
       'Watford', 'West Ham', 'Crystal Palace', 'Man City', 'Liverpool'],
      dtype=object)

In [17]:
# calculate average goals of all league two arguments of a data frame and num of games returns a lists
def calc_lge_avg(df, num_games = 23):
    team_count = len(df['HomeTeam'].unique())
    games = df.tail(num_games * team_count)
    homeMean = games['FTHG'].mean()
    awayMean = games['FTAG'].mean()
    return [homeMean, awayMean]
    

In [18]:
print(calc_lge_avg(data, num_games = 23))

[1.4921052631578948, 1.2078947368421054]


In [19]:
# get teams last number of games four arguments team name, home or away, num of games and dataframe 
def team_last_games(team, home_away, num_of_games, df):
    games = df[(df[home_away]==team)].tail(num_of_games)
    return games
    

In [20]:
# get teams attacking and defensive strengths as a list value at index 1 is attacking strengths
# and defensive strength at index 2
def team_strengths(team, home_away, num_of_games, df):
    df2 = team_last_games(team, home_away, num_of_games, df)
    lg_avg = calc_lge_avg(df, num_of_games)
    tm_avg = 0
    stList = []
    if home_away == 'HomeTeam':
        stList.append((df2['FTHG'].mean()) / lg_avg[0])
        stList.append((df2['FTAG'].mean()) / lg_avg[1])
    elif home_away == 'AwayTeam':
        stList.append((df2['FTAG'].mean()) / lg_avg[1])
        stList.append((df2['FTHG'].mean()) / lg_avg[0])
    
    return stList
        
    

In [21]:
print(team_strengths('Stoke', 'AwayTeam', 19, data))

[0.8278867102396513, 1.09347442680776]


In [22]:
print(team_strengths('Tottenham', 'HomeTeam', 19, data))

[1.2345679012345678, 0.6535947712418301]


In [23]:
# calculate each teams expected goals
def exp_goals(home, away, num_of_games, data):
    home_str =  team_strengths(home, 'HomeTeam', num_of_games, data)
    away_str = team_strengths(away,'AwayTeam', num_of_games, data)
    lg_avg = lg_avg = calc_lge_avg(data, num_of_games)
    
    home_exp = round(home_str[0] * away_str[1] * lg_avg[0], 3)
    away_exp = round(away_str[0] * home_str[1] * lg_avg[1], 3)
    
    return [home_exp, away_exp]
    

In [24]:
print(exp_goals('Tottenham', 'Stoke',lg_games , data))

[2.014, 0.654]


In [25]:
# calculates probability scoring an amount of goals using possion Probability mass function
def goals_prob(home, away, num_games, df):
    poss = []
    goals = [0, 1, 2, 3, 4, 5, 6]
    expected = exp_goals(home, away, num_games, df)
    for i in expected:
        if expected.index(i) == 0:
            aList = []
            for g in goals:
                aList.append(round(poisson.pmf(g, i) * 100, 2))
        elif expected.index(i) == 1:
            aList = []
            for g in goals:
                aList.append(round(poisson.pmf(g, i) * 100, 2))
        poss.append(aList)
    return poss
    

In [26]:
print(goals_prob('Tottenham', 'Stoke', lg_games, data))

[[13.35, 26.88, 27.07, 18.17, 9.15, 3.69, 1.24], [52.0, 34.01, 11.12, 2.42, 0.4, 0.05, 0.01]]
